# Predicting high-potential FIFA players using individual performance data
Merete Lutz, Jake Barnabe, Simon Frew, Waleed Mahmood

DSCI 522, Group 17

## Introduction

## Methods

### Data

### Analysis

## Results & Discussion

In [135]:
import warnings

# Ignore FutureWarnings
warnings.simplefilter(action='ignore', category=FutureWarning)


import os 
import requests
import warnings
import zipfile

import numpy as np
import pandas as pd
import altair as alt


from hashlib import sha1
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import (
    GridSearchCV,
    RandomizedSearchCV,
    cross_validate,
    train_test_split,
)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB, GaussianNB
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

warnings.filterwarnings('ignore', category=FutureWarning)
alt.data_transformers.enable("vegafusion");
pd.set_option('display.max_rows', 200)

In [2]:
# download dataset
# method adapted from: https://github.com/ttimbers/breast_cancer_predictor_py/tree/0.0.1
url = "https://sports-statistics.com/database/fifa/fifa_2022_datasets.zip"

request = requests.get(url)

with open(os.path.join("data", "fifa_2022_datasets.zip"), "wb") as f: 
    f.write(request.content)
with zipfile.ZipFile(os.path.join("data", "fifa_2022_datasets.zip"), "r") as zip_file: 
    zip_file.extract("players_22.csv", path="data")
os.remove(os.path.join("data", "fifa_2022_datasets.zip"))

In [141]:
df

,wage_eur,age,height_cm,weight_kg,weak_foot,skill_moves,pace,shooting,passing,dribbling,defending,physic,target
0,320000.0,34,170,72,4,4,85.0,92.0,91.0,95.0,34.0,65.0,Great
1,270000.0,32,185,81,4,4,78.0,92.0,79.0,86.0,44.0,82.0,Great
2,270000.0,36,187,83,4,5,87.0,94.0,80.0,88.0,34.0,75.0,Great
3,270000.0,29,175,68,5,5,91.0,83.0,86.0,94.0,37.0,63.0,Great
4,350000.0,30,181,70,5,4,76.0,86.0,93.0,88.0,64.0,78.0,Great
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19234,1000.0,22,180,64,3,2,58.0,35.0,46.0,48.0,42.0,49.0,Low
19235,500.0,19,175,70,3,2,59.0,39.0,50.0,46.0,41.0,51.0,Low
19236,500.0,21,178,72,3,2,60.0,37.0,45.0,49.0,41.0,52.0,Low
19237,500.0,19,173,66,3,2,68.0,46.0,36.0,48.0,15.0,42.0,Low


In [155]:
# data selection
df = pd.read_csv("data/players_22.csv", encoding="utf-8", low_memory=False)

df

,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,...,lcb,cb,rcb,rb,gk,player_face_url,club_logo_url,club_flag_url,nation_logo_url,nation_flag_url
0,158023,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,Lionel Andrés Messi Cuccittini,"RW, ST, CF",93,93,78000000.0,320000.0,34,...,50+3,50+3,50+3,61+3,19+3,https://cdn.sofifa.net/players/158/023/22_120.png,https://cdn.sofifa.net/teams/73/60.png,https://cdn.sofifa.net/flags/fr.png,https://cdn.sofifa.net/teams/1369/60.png,https://cdn.sofifa.net/flags/ar.png
1,188545,https://sofifa.com/player/188545/robert-lewand...,R. Lewandowski,Robert Lewandowski,ST,92,92,119500000.0,270000.0,32,...,60+3,60+3,60+3,61+3,19+3,https://cdn.sofifa.net/players/188/545/22_120.png,https://cdn.sofifa.net/teams/21/60.png,https://cdn.sofifa.net/flags/de.png,https://cdn.sofifa.net/teams/1353/60.png,https://cdn.sofifa.net/flags/pl.png
2,20801,https://sofifa.com/player/20801/c-ronaldo-dos-...,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"ST, LW",91,91,45000000.0,270000.0,36,...,53+3,53+3,53+3,60+3,20+3,https://cdn.sofifa.net/players/020/801/22_120.png,https://cdn.sofifa.net/teams/11/60.png,https://cdn.sofifa.net/flags/gb-eng.png,https://cdn.sofifa.net/teams/1354/60.png,https://cdn.sofifa.net/flags/pt.png
3,190871,https://sofifa.com/player/190871/neymar-da-sil...,Neymar Jr,Neymar da Silva Santos Júnior,"LW, CAM",91,91,129000000.0,270000.0,29,...,50+3,50+3,50+3,62+3,20+3,https://cdn.sofifa.net/players/190/871/22_120.png,https://cdn.sofifa.net/teams/73/60.png,https://cdn.sofifa.net/flags/fr.png,NaN,https://cdn.sofifa.net/flags/br.png
4,192985,https://sofifa.com/player/192985/kevin-de-bruy...,K. De Bruyne,Kevin De Bruyne,"CM, CAM",91,91,125500000.0,350000.0,30,...,69+3,69+3,69+3,75+3,21+3,https://cdn.sofifa.net/players/192/985/22_120.png,https://cdn.sofifa.net/teams/10/60.png,https://cdn.sofifa.net/flags/gb-eng.png,https://cdn.sofifa.net/teams/1325/60.png,https://cdn.sofifa.net/flags/be.png
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19234,261962,https://sofifa.com/player/261962/defu-song/220002,Song Defu,宋德福,CDM,47,52,70000.0,1000.0,22,...,46+2,46+2,46+2,48+2,15+2,https://cdn.sofifa.net/players/261/962/22_120.png,https://cdn.sofifa.net/teams/112541/60.png,https://cdn.sofifa.net/flags/cn.png,NaN,https://cdn.sofifa.net/flags/cn.png
19235,262040,https://sofifa.com/player/262040/caoimhin-port...,C. Porter,Caoimhin Porter,CM,47,59,110000.0,500.0,19,...,44+2,44+2,44+2,48+2,14+2,https://cdn.sofifa.net/players/262/040/22_120.png,https://cdn.sofifa.net/teams/445/60.png,https://cdn.sofifa.net/flags/ie.png,NaN,https://cdn.sofifa.net/flags/ie.png
19236,262760,https://sofifa.com/player/262760/nathan-logue/...,N. Logue,Nathan Logue-Cunningham,CM,47,55,100000.0,500.0,21,...,45+2,45+2,45+2,47+2,12+2,https://cdn.sofifa.net/players/262/760/22_120.png,https://cdn.sofifa.net/teams/111131/60.png,https://cdn.sofifa.net/flags/ie.png,NaN,https://cdn.sofifa.net/flags/ie.png
19237,262820,https://sofifa.com/player/262820/luke-rudden/2...,L. Rudden,Luke Rudden,ST,47,60,110000.0,500.0,19,...,26+2,26+2,26+2,32+2,15+2,https://cdn.sofifa.net/players/262/820/22_120.png,https://cdn.sofifa.net/teams/111131/60.png,https://cdn.sofifa.net/flags/ie.png,NaN,https://cdn.sofifa.net/flags/ie.png


In [156]:
df = df.loc[:, ["potential",
                #"player_positions",
                "value_eur",
        "wage_eur", 
        "age", 
        "height_cm", 
        "weight_kg", 
        "weak_foot", 
        "skill_moves", 
        'pace',
        'shooting',
        'passing',
        'dribbling',
        'defending',
        'physic',]]

df = df.dropna()

In [152]:
df.describe()

,potential,wage_eur,age,height_cm,weight_kg,weak_foot,skill_moves,pace,shooting,passing,dribbling,defending,physic
count,17041.000000,17041.000000,17041.000000,17041.000000,17041.000000,17041.000000,17041.000000,17041.000000,17041.000000,17041.000000,17041.000000,17041.000000,17041.000000
mean,71.243061,9354.732703,25.065959,180.389179,74.089314,3.000998,2.520451,68.235491,52.327446,57.298163,62.556716,51.691098,64.812628
std,6.048461,19880.724994,4.605071,6.545547,6.696454,0.649710,0.637406,10.905376,14.052085,10.068871,9.653533,16.185957,9.792269
min,49.000000,500.000000,16.000000,155.000000,49.000000,1.000000,2.000000,28.000000,18.000000,25.000000,27.000000,14.000000,29.000000
25%,67.000000,1000.000000,21.000000,176.000000,70.000000,3.000000,2.000000,62.000000,42.000000,51.000000,57.000000,37.000000,59.000000
50%,71.000000,3000.000000,25.000000,180.000000,74.000000,3.000000,2.000000,69.000000,54.000000,58.000000,64.000000,56.000000,66.000000
75%,75.000000,9000.000000,28.000000,185.000000,78.000000,3.000000,3.000000,76.000000,63.000000,64.000000,69.000000,64.000000,72.000000
max,95.000000,350000.000000,39.000000,203.000000,110.000000,5.000000,5.000000,97.000000,94.000000,93.000000,95.000000,91.000000,90.000000


In [157]:
df['target'] = pd.cut(x=df['potential'], bins=[0, 67, 71, 75, 100], 
                     labels=['Low', 'Medium', 'Good', 'Great'])

In [149]:
df

,potential,player_positions,value_eur,wage_eur,age,height_cm,weight_kg,weak_foot,skill_moves,pace,shooting,passing,dribbling,defending,physic,target
0,93,"RW, ST, CF",78000000.0,320000.0,34,170,72,4,4,85.0,92.0,91.0,95.0,34.0,65.0,Great
1,92,ST,119500000.0,270000.0,32,185,81,4,4,78.0,92.0,79.0,86.0,44.0,82.0,Great
2,91,"ST, LW",45000000.0,270000.0,36,187,83,4,5,87.0,94.0,80.0,88.0,34.0,75.0,Great
3,91,"LW, CAM",129000000.0,270000.0,29,175,68,5,5,91.0,83.0,86.0,94.0,37.0,63.0,Great
4,91,"CM, CAM",125500000.0,350000.0,30,181,70,5,4,76.0,86.0,93.0,88.0,64.0,78.0,Great
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19234,52,CDM,70000.0,1000.0,22,180,64,3,2,58.0,35.0,46.0,48.0,42.0,49.0,Low
19235,59,CM,110000.0,500.0,19,175,70,3,2,59.0,39.0,50.0,46.0,41.0,51.0,Low
19236,55,CM,100000.0,500.0,21,178,72,3,2,60.0,37.0,45.0,49.0,41.0,52.0,Low
19237,60,ST,110000.0,500.0,19,173,66,3,2,68.0,46.0,36.0,48.0,15.0,42.0,Low


In [158]:
df = df.drop(['potential'], axis=1)

In [159]:
train_df, test_df = train_test_split(df, test_size=0.3, random_state=123)
train_df

,value_eur,wage_eur,age,height_cm,weight_kg,weak_foot,skill_moves,pace,shooting,passing,dribbling,defending,physic,target
11022,1800000.0,1000.0,19,172,68,3,2,68.0,48.0,67.0,66.0,59.0,57.0,Great
5994,2100000.0,22000.0,24,185,83,3,3,74.0,67.0,65.0,73.0,36.0,60.0,Good
8739,350000.0,1000.0,35,162,64,4,3,50.0,61.0,70.0,70.0,26.0,41.0,Low
16553,200000.0,650.0,28,180,76,3,2,55.0,45.0,50.0,53.0,57.0,70.0,Low
13883,550000.0,1000.0,25,180,76,2,2,70.0,48.0,54.0,60.0,59.0,62.0,Low
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112,27000000.0,84000.0,33,190,79,4,5,75.0,86.0,84.0,86.0,40.0,67.0,Great
14943,800000.0,2000.0,20,175,68,3,3,70.0,53.0,55.0,67.0,31.0,48.0,Great
8609,1100000.0,500.0,27,172,70,3,2,79.0,64.0,60.0,68.0,24.0,47.0,Low
17201,240000.0,2000.0,24,183,76,2,2,54.0,33.0,40.0,43.0,55.0,66.0,Low


In [160]:
df_numeric = df.select_dtypes(include = ['int64', 'float64'])
df_numeric_list = df_numeric.columns
print(df_numeric_list)
# initialize an empty list to store the distribution of each numeric column
charts_numeric = []

# create a visualization of the distribution of each numeric column and store it in the `charts` list
for column in df_numeric_list:
    chart = alt.Chart(df).mark_bar().encode(
        alt.X(column, bin=alt.Bin(maxbins=20), title=f'{column}'),
        alt.Y('count()', title='Frequency')
    ).properties(
        width=150,
        height=100,
        title=f'Distribution of {column}'
    )
    charts_numeric.append(chart)
length = len(charts_numeric)
# # Create a 4x4 grid with all the distributions
# alt.vconcat(
#     alt.hconcat(*charts_numeric[:round(length/4)]),
#     alt.hconcat(*charts_numeric[round(length/4):round(length/2)]),
#     alt.hconcat(*charts_numeric[round(length/2):round(3*length/4)]),
#     alt.hconcat(*charts_numeric[round(3*length/4):length-1])
# ).configure_title(fontSize=10).configure_axis(titleFontSize=8)

alt.vconcat(
    alt.hconcat(*charts_numeric[:4]),
    alt.hconcat(*charts_numeric[4:8]),
    alt.hconcat(*charts_numeric[8:12]),
    alt.hconcat(*charts_numeric[12:16])
).configure_title(fontSize=10).configure_axis(titleFontSize=8)

Index(['value_eur', 'wage_eur', 'age', 'height_cm', 'weight_kg', 'weak_foot',
       'skill_moves', 'pace', 'shooting', 'passing', 'dribbling', 'defending',
       'physic'],
      dtype='object')


alt.VConcatChart(...)

In [161]:
X_train = train_df.drop(columns = ['target'])
y_train = train_df['target']
X_test = test_df.drop(columns = ['target'])
y_test = test_df['target']

In [163]:
# BELOW THIS IS JUST COPY AND PASTED IN FROM THE LAB 4

In [164]:
models = {
    "dummy": DummyClassifier(random_state=123), 
    "Decision Tree": DecisionTreeClassifier(random_state=123),
    "KNN": KNeighborsClassifier(),
    "RBF SVM": SVC(random_state=123),
    "Naive Bayes": GaussianNB(),
    "Logistic Regression": LogisticRegression(max_iter=2000, multi_class="ovr", random_state=123),
}

In [165]:
# From Leture 2 of 571 at the below link
# https://pages.github.ubc.ca/MDS-2023-24/DSCI_571_sup-learn-1_students/lectures/02_ml-fundamentals.html?highlight=mean_std_cross_val_scores
def mean_std_cross_val_scores(model, X_train, y_train, **kwargs):
    """
    Returns mean and std of cross validation
    """
    scores = cross_validate(model, X_train, y_train, **kwargs)

    mean_scores = pd.DataFrame(scores).mean()
    std_scores = pd.DataFrame(scores).std()
    out_col = []

    for i in range(len(mean_scores)):
        out_col.append((f"%0.3f (+/- %0.3f)" % (mean_scores[i], std_scores[i])))

    return pd.Series(data=out_col, index=mean_scores.index)

In [166]:
results = pd.DataFrame()
for name, model in models.items():
    pipe = make_pipeline(StandardScaler(),
                         model)
    results[name] = mean_std_cross_val_scores(pipe, X_train, y_train, return_train_score=True)

In [167]:
results

,dummy,Decision Tree,KNN,RBF SVM,Naive Bayes,Logistic Regression
fit_time,0.004 (+/- 0.000),0.037 (+/- 0.001),0.008 (+/- 0.000),1.138 (+/- 0.013),0.007 (+/- 0.000),0.059 (+/- 0.004)
score_time,0.002 (+/- 0.001),0.002 (+/- 0.000),0.129 (+/- 0.002),0.544 (+/- 0.011),0.001 (+/- 0.000),0.002 (+/- 0.000)
test_score,0.276 (+/- 0.000),0.821 (+/- 0.005),0.566 (+/- 0.005),0.751 (+/- 0.008),0.570 (+/- 0.007),0.705 (+/- 0.005)
train_score,0.276 (+/- 0.000),1.000 (+/- 0.000),0.727 (+/- 0.004),0.786 (+/- 0.003),0.570 (+/- 0.002),0.708 (+/- 0.002)


In [168]:
from scipy.stats import loguniform, randint, uniform
from sklearn.model_selection import RandomizedSearchCV

param_dist = {
    "svc__C": [0.001, 0.01, 0.1, 1, 10, 100],
    "svc__gamma": [0.001, 0.01, 0.1, 1, 10, 100]
}
pipe = make_pipeline(StandardScaler(),
                     SVC(random_state=123))

random_search = RandomizedSearchCV(pipe,
                                   param_dist, 
                                   n_iter=10, 
                                   n_jobs=-1, 
                                   return_train_score=True,
                                   random_state=123)
                                   

random_search.fit(X_train, y_train)

pd.DataFrame(random_search.cv_results_)[
    [
        "mean_test_score",
        "mean_train_score",        
        "param_svc__C",
        "param_svc__gamma",
        "mean_fit_time",
        "rank_test_score",
    ]
].set_index("rank_test_score").sort_index().iloc[:5]

/Users/meretelutz/miniconda3/envs/573/lib/python3.11/site-packages/sklearn/utils/validation.py:757: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Users/meretelutz/miniconda3/envs/573/lib/python3.11/site-packages/sklearn/utils/validation.py:757: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Users/meretelutz/miniconda3/envs/573/lib/python3.11/site-packages/sklearn/utils/validation.py:757: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Users/meretelutz/miniconda3/envs/573/lib/python3.11/site-packages/sklearn/utils/v

,mean_test_score,mean_train_score,param_svc__C,param_svc__gamma,mean_fit_time
rank_test_score,,,,,
1,0.811619,0.836016,100,0.01,1.608477
2,0.779175,0.948462,100,0.1,3.076319
3,0.567739,0.571911,0.1,0.01,2.864958
4,0.559775,0.563548,1,0.001,2.586529
5,0.498658,0.505554,0.01,0.1,3.679393


In [169]:
results

,dummy,Decision Tree,KNN,RBF SVM,Naive Bayes,Logistic Regression
fit_time,0.004 (+/- 0.000),0.037 (+/- 0.001),0.008 (+/- 0.000),1.138 (+/- 0.013),0.007 (+/- 0.000),0.059 (+/- 0.004)
score_time,0.002 (+/- 0.001),0.002 (+/- 0.000),0.129 (+/- 0.002),0.544 (+/- 0.011),0.001 (+/- 0.000),0.002 (+/- 0.000)
test_score,0.276 (+/- 0.000),0.821 (+/- 0.005),0.566 (+/- 0.005),0.751 (+/- 0.008),0.570 (+/- 0.007),0.705 (+/- 0.005)
train_score,0.276 (+/- 0.000),1.000 (+/- 0.000),0.727 (+/- 0.004),0.786 (+/- 0.003),0.570 (+/- 0.002),0.708 (+/- 0.002)


In [170]:
best_model = random_search.best_estimator_
best_model

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(C=100, gamma=0.01, random_state=123))])

In [171]:
best_model.score(X_test, y_test)

0.8089184431840407

## References